In [117]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [118]:
data=pd.read_csv("data.csv")

In [119]:
data.head()

,Time(Sec),Unnamed: 1,Node Number,Temperature,Humidity,CO,CO2,State of the system
0,1,2019-12-26 15:20:09.849088,1,13.77,59.11,3.76,838.42,1
1,2,2019-12-26 15:20:12.383318,1,13.75,59.54,3.76,833.53,1
2,3,2019-12-26 15:20:14.652292,1,13.70,59.80,3.68,823.75,1
3,4,2019-12-26 15:20:16.819458,1,13.63,59.46,3.68,828.64,1
4,5,2019-12-26 15:20:21.815297,1,13.56,58.98,3.59,828.64,1


In [120]:
data.isna().sum()


Time(Sec)              0
Unnamed: 1             1
Node Number            0
Temperature            0
Humidity               0
CO                     0
CO2                    0
State of the system    0
dtype: int64

In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Time(Sec)            80 non-null     int64  
 1   Unnamed: 1           79 non-null     object 
 2   Node Number          80 non-null     int64  
 3   Temperature          80 non-null     float64
 4   Humidity             80 non-null     float64
 5   CO                   80 non-null     float64
 6   CO2                  80 non-null     float64
 7   State of the system  80 non-null     int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 5.1+ KB


In [122]:
data=data.drop("Unnamed: 1",axis=1)

In [123]:
x=data.drop("State of the system",axis=1)
y=data["State of the system"]

In [124]:
from sklearn.model_selection import train_test_split

In [125]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [126]:
from sklearn.neighbors import KNeighborsClassifier 
np.random.seed(42)
knn = KNeighborsClassifier(n_neighbors = 7).fit(x_train, y_train)
knn_predictions=knn.predict(x_test)
knn.score(x_test,y_test)

0.85

In [111]:
#hyperparameter tuning
np.random.seed(42)
from sklearn.model_selection import GridSearchCV
leaf_size = list(range(1,50,2))
n_neighbors = list(range(1,30,3))
p=[1,2]
knn_2 = KNeighborsClassifier()

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
clf = GridSearchCV(knn_2, hyperparameters, cv=5)
best_model = clf.fit(x_train,y_train)
clf.score(x_train,y_train)


0.9833333333333333

In [113]:
from sklearn.naive_bayes import GaussianNB
np.random.seed(42)
gnb = GaussianNB().fit(x_train, y_train) 
gnb_predictions = gnb.predict(x_test) 
gnb.score(x_test,y_test)

0.85

In [114]:
#hyperparameter tuning 
np.random.seed(42)

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

gs_NB = GridSearchCV(estimator=nb_classifier, 
                     param_grid=params_NB, 
                     cv=5,
                     verbose=1, 
                     scoring='accuracy')

gs_NB.fit(x_test,y_test)
gs_NB.score(x_train,y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


E:\ML-projects\csir_dataset\env\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.7s finished


0.85